In [1]:
import sys
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import cvxpy as cp
from collections import defaultdict

# Add custom paths
sys.path.insert(0, 'E:\\myREPO\\Fairness-Decision-Focused-Loss\\fold-opt-package\\fold_opt')

# Suppress warnings
warnings.filterwarnings("ignore")

from GMRES import *
from fold_opt import *

# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

from src.utils.myOptimization import (
    AlphaFairnesstorch,
    solveIndProblem, solve_closed_form, solve_group, solve_group_grad,
    compute_coupled_group_obj
)
from src.utils.myPrediction import generate_random_features, customPredictionModel
from src.utils.plots import visLearningCurve
from src.fairness.cal_fair_penalty import atkinson_loss, mean_abs_dev
from src.utils.features import get_all_features


Using device: cpu


In [2]:
def compute_coupled_group_obj_torch(d, b, group_idx, alpha, beta=None):
    """
    Calculates the objective value for the new alpha=beta formulation using torch tensors.
    """
    # Ensure all inputs are torch tensors
    if not torch.is_tensor(d):
        d = torch.tensor(d, dtype=torch.float32)
    if not torch.is_tensor(b):
        b = torch.tensor(b, dtype=torch.float32)
    if not torch.is_tensor(group_idx):
        group_idx = torch.tensor(group_idx, dtype=torch.float32)

    d = d.reshape(-1)
    b = b.reshape(-1)
    group_idx = group_idx.reshape(-1)

    epsilon = 1e-12
    y = b * d + epsilon
    unique_groups = torch.unique(group_idx)
    g_k_values = torch.zeros(len(unique_groups), dtype=torch.float32, device=y.device)

    for i, k in enumerate(unique_groups):
        members_mask = (group_idx == k)
        y_k = y[members_mask]
        if 0 < alpha < 1:
            g_k_values[i] = torch.sum(y_k ** (1 - alpha)) / (1 - alpha)
        elif alpha > 1:
            g_k_values[i] = (alpha - 1) / torch.sum(y_k ** (1 - alpha))
        elif abs(alpha - 1.0) < epsilon:
            g_k_values[i] = torch.sum(torch.log(y_k))
        else:  # alpha <= 0
            g_k_values[i] = torch.sum(y_k)

    if alpha == float('inf') or str(alpha).lower() == 'inf':
        objective_value = torch.min(g_k_values)
    elif abs(alpha - 1.0) < epsilon:
        objective_value = torch.sum(torch.log(g_k_values + epsilon))
    else:
        objective_value = torch.sum(g_k_values)

    return objective_value

def alpha_fair_individual(utilities, alpha):
    """
    Calculate alpha-fair objective for individual optimization.
    Works with both numpy arrays and torch tensors.
    """
    if isinstance(utilities, torch.Tensor):
        utilities = utilities.detach().cpu().numpy()
    
    utilities = np.maximum(utilities, 1e-8)  # Avoid log(0) or division by 0
    
    if alpha == 0.5:
        return 2 * np.sqrt(np.sum(utilities))
    elif alpha == 1.0:
        return np.log(np.sum(utilities))
    elif alpha == float('inf') or str(alpha).lower() == 'inf':
        return np.min(utilities)
    else:
        return (np.sum(utilities) ** alpha) / alpha

## Define Alpha & Q

In [3]:
alpha, Q = 1.5, 1000

In [4]:
def to_numpy_1d(x):
    """Return a 1-D NumPy array; error if the length is not > 1."""
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    x = np.asarray(x).reshape(-1)
    assert x.ndim == 1, f"expected 1-D, got shape {x.shape}"
    return x


In [5]:
# df = pd.read_csv('/Users/dennis/Downloads/2024-fall/research/Fairness-Decision-Focused-Loss/Organized-FDFL/src/data/data.csv')

df = pd.read_csv(r'E:\myREPO\Fairness-Decision-Focused-Loss\Organized-FDFL\src\data\data.csv')

df = df.sample(n=5000, random_state=42)

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# ---------- basic 1-D helpers ----------
def as_1d(a, dtype=np.float32):
    a = np.asarray(a, dtype=dtype).reshape(-1)   # (N,)
    if a.ndim != 1:
        raise ValueError(f"expect 1-D, got {a.shape}")
    return a

# transform the features
scaler = StandardScaler()

# risk   = as_1d(df['risk_score_t']) * 100
risk = np.array(df['benefit']) * 100.0  # scale to [0,100]
risk = np.maximum(risk,1) + 1
gainF  = np.ones_like(risk, dtype=np.float32)
cost   = as_1d(df['cost_t_capped']) * 10.0
cost   = np.maximum(cost, 1)              # keep strictly positive
race   = as_1d(df['race'])  # keep as int

feats  = scaler.fit_transform(df[get_all_features(df)]).astype(np.float32)   # (N,p)



In [6]:
class optDataset(Dataset):
    def __init__(self, optmodel, feats, risk, gainF, cost, race, alpha=alpha, Q=Q):
        # Store as numpy arrays for now
        self.feats = feats
        self.risk = risk
        self.gainF = gainF
        self.cost = cost
        self.race = race
        self.optmodel = optmodel

        # Call optmodel (expects numpy arrays)
        sol = self.optmodel(self.risk, self.cost, self.race, Q=Q, alpha=alpha)
        obj = compute_coupled_group_obj_torch(sol, self.risk, self.race, alpha=alpha)

        # Convert everything to torch tensors for storage
        self.feats = torch.from_numpy(self.feats).float()
        self.risk = torch.from_numpy(self.risk).float()
        self.gainF = torch.from_numpy(self.gainF).float()
        self.cost = torch.from_numpy(self.cost).float()
        self.race = torch.from_numpy(self.race).float()
        self.sol = torch.from_numpy(sol).float()
        self.obj = torch.tensor(obj).float()

    def __len__(self):
        return len(self.feats)

    # def __getitem__(self, idx):
    #     return self.feats, self.risk, self.gainF, self.cost, self.race, self.sol, self.obj

    def __getitem__(self, idx):
        
        return ( self.feats[idx],
                self.risk[idx],
                self.gainF[idx],
                self.cost[idx],
                self.race[idx],
                self.sol[idx],    # or store per-item solutions; see note ▼
                self.obj )  


## Prediction Model

In [7]:
class FairRiskPredictor(nn.Module):
    """
    An improved predictor model featuring Batch Normalization for stability
    and Kaiming (He) weight initialization for faster convergence.
    """
    def __init__(self, input_dim, dropout_rate=0.2, hidden_dim=64):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 1),
            nn.Softplus()
        )
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.model:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # The squeeze operation is now done in the training loop for clarity
        return self.model(x).squeeze(-1)

# Load Data

In [8]:
# Setup training parameters

optmodel = solve_group

feats_np  = np.asarray(feats)              # 2-D OK
gainF_np  = to_numpy_1d(gainF)
risk_np   = to_numpy_1d(risk)
cost_np   = to_numpy_1d(cost)
race_np   = to_numpy_1d(df['race'].values)


# Perform train-test split
feats_train, feats_test, gainF_train, gainF_test, risk_train, risk_test, cost_train, cost_test, race_train, race_test = train_test_split(
    feats, gainF, risk, cost, df['race'].values, test_size=0.5, random_state=2
)

print(f"Train size: {feats_train.shape[0]}")
print(f"Test size: {feats_test.shape[0]}")

dataset_train = optDataset(optmodel, feats_train, risk_train, gainF_train, cost_train, race_train, alpha=alpha, Q=Q)
dataset_test = optDataset(optmodel, feats_test, risk_test, gainF_test, cost_test, race_test, alpha=alpha, Q=Q)

# Create dataloaders
dataloader_train = DataLoader(dataset_train, batch_size=len(dataset_train), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_train), shuffle=False)

predmodel = FairRiskPredictor(feats_train.shape[1])
predmodel.to(DEVICE)
# save the initial model
# torch.save(predmodel.state_dict(), 'initial_model.pth')
# load the initial model

# self.sol is (N,) – __getitem__ returns a scalar component;
# DataLoader stacks to (B,), which is exactly what the training loop expects.


Train size: 2500
Test size: 2500


FairRiskPredictor(
  (model): Sequential(
    (0): Linear(in_features=152, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Softplus(beta=1.0, threshold=20.0)
  )
)

# Alpha Fair Obj

In [9]:
import numpy as np
import torch

def alpha_fair_individual(u, alpha, epsilon=1e-12):
    """
    Calculates the alpha-fairness objective for individuals.
    This function is type-aware and works with both PyTorch Tensors and NumPy arrays.
    """
    # Check the input type to use the correct library
    is_torch = isinstance(u, torch.Tensor)

    # Add epsilon for numerical stability
    u = u + epsilon

    if abs(alpha - 1.0) < epsilon:
        return torch.log(u).sum(-1) if is_torch else np.log(u).sum(-1)
    
    elif abs(alpha - 0.0) < epsilon:
        return u.sum(-1)

    elif alpha == float('inf'):
        if is_torch:
            return u.min(-1).values
        else:
            return u.min(-1)

    # Use the appropriate power function based on the type
    if is_torch:
        return (torch.pow(u, 1 - alpha) / (1 - alpha)).sum(-1)
    else:
        return (np.power(u, 1 - alpha) / (1 - alpha)).sum(-1)

In [10]:
def solve_optimization(gainF, risk, cost, alpha, Q):
    gainF = gainF.detach().cpu().numpy() if isinstance(gainF, torch.Tensor) else gainF
    risk = risk.detach().cpu().numpy() if isinstance(risk, torch.Tensor) else risk
    cost = cost.detach().cpu().numpy() if isinstance(cost, torch.Tensor) else cost

    risk = risk.clip(1)
    gainF, risk, cost = gainF.flatten(), risk.flatten(), cost.flatten()
    d = cp.Variable(risk.shape, nonneg=True)

    if gainF.shape != risk.shape or risk.shape != cost.shape:
        raise ValueError("Dimensions of gainF, risk, and cost do not match")

    utils = cp.multiply(cp.multiply(gainF, risk), d)
    constraints = [d >= 0, cp.sum(cost * d) <= Q]

    if alpha == 'inf':
        t = cp.Variable()
        objective = cp.Maximize(t)
        constraints.append(utils >= t)
    elif alpha == 1:
        objective = cp.Maximize(cp.sum(cp.log(utils)))
    elif alpha == 0:
        objective = cp.Maximize(cp.sum(utils))
    else:
        objective = cp.Maximize(cp.sum(utils**(1-alpha)) / (1-alpha))

    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=False, warm_start=True, mosek_params={'MSK_IPAR_LOG': 1})

    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")

    optimal_decision = d.value
    optimal_value = alpha_fair_individual(optimal_decision * gainF * risk, alpha)

    return optimal_decision, optimal_value

# Analytical Projection

In [11]:
def proj_budget(x, cost, Q, max_iter=100):
    """
    x : (B,n)   or (n,)   –– internally promoted to (B,n)
    cost : (n,) positive
    Q : scalar or length‑B tensor
    """
    batched = x.dim() == 2
    if not batched:                       # (n,)  →  (1,n)
        x = x.unsqueeze(0)

    B, n = x.shape
    cost = cost.to(x)
    Q    = torch.as_tensor(Q, dtype=x.dtype, device=x.device).reshape(-1, 1)  # (B,1)

    d    = x.clamp(min=0.)                # enforce non‑neg
    viol = (d @ cost) > Q.squeeze(1)      # which rows violate the budget?

    if viol.any():
        dv, Qv = d[viol], Q[viol]
        lam_lo = torch.zeros_like(Qv.squeeze(1))
        lam_hi = (dv / cost).max(1).values   # upper bound for λ⋆

        for _ in range(max_iter):
            lam_mid = 0.5 * (lam_lo + lam_hi)
            trial   = (dv - lam_mid[:, None] * cost).clamp(min=0.)
            too_big = (trial @ cost) > Qv.squeeze(1)
            lam_lo[too_big] = lam_mid[too_big]
            lam_hi[~too_big]= lam_mid[~too_big]

        d[viol] = (dv - lam_hi[:, None] * cost).clamp(min=0.)

    return d if batched else d.squeeze(0)   # restore original rank


# Cvxpylayer Projection

In [12]:
from cvxpylayers.torch.cvxpylayer import CvxpyLayer
# --- 1. Define the Differentiable Projection Layer ---
def get_differentiable_projection(n, cost_np, Q_val):
    """
    Creates a differentiable projection layer using CvxpyLayer.
    This layer solves the projection problem:
        minimize   ||d - z||_2^2
        subject to cost^T * d <= Q
                   d >= 0
    """
    # Define CVXPY variables and parameters
    d_var = cp.Variable(n)
    z_param = cp.Parameter(n)
    objective = cp.Minimize(cp.sum_squares(d_var - z_param))
    constraints = [cost_np @ d_var <= Q_val, d_var >= 0]
    problem = cp.Problem(objective, constraints)
    return CvxpyLayer(problem, parameters=[z_param], variables=[d_var])


# Fold-Opt Layer

In [13]:
def pgd_step_cvxpylayer(r, d, g, race, cost, Q, alpha, lr, projection_layer, group=False):
    """
    Performs one PGD step using the CvxpyLayer for projection.
    """
    # Temporarily enable gradients for the internal gradient calculation
    with torch.enable_grad():
        # The input 'd' comes from the solver and has no grad history.
        # We clone it and require gradients to trace the PGD update.
        d_clone = d.clone().requires_grad_(True)
    
        if not group:
            obj = alpha_fair_individual(g* d_clone * r, alpha=alpha)
        else:
            obj = compute_coupled_group_obj_torch(d_clone, r, race, alpha=alpha)
        
        # Compute gradient of the objective w.r.t. the decision `d_clone`
        # This now happens within the `enable_grad` context.
        grad_d, = torch.autograd.grad(obj, d_clone, create_graph=True)

    # Perform the gradient ascent step
    unprojected_d = d + lr * grad_d
    
    # Project back to the feasible set using the differentiable layer
    projected_d, = projection_layer(unprojected_d)
    
    return projected_d

def make_foldopt_layer_cvxpylayer(g, cost, race, alpha, Q, group, lr=5e-3, n_fixedpt=50, backprop_rule='FPI'):
    """
    Factory function to create a FoldOptLayer with a differentiable PGD update.
    """
    # Detach tensors that are not parameters of the prediction model but are used in the optimization.
    # This prevents trying to compute gradients with respect to them.
    g_detached = g.detach()
    cost_detached = cost.detach()
    race_detached = race.detach()

    # --- Create the differentiable projection layer once ---
    n_vars = cost.shape[0]
    cost_np = cost_detached.cpu().numpy()
    Q_val = Q.item() if isinstance(Q, torch.Tensor) else Q
    projection_layer = get_differentiable_projection(n_vars, cost_np, Q_val)

    # --- Solver function (no gradients needed) ---
    def solver_fn(r_b): # r_b is batched, expected shape (B, n)
        # Assuming the solver can only handle one instance at a time.
        # If your solver is batched, you can simplify this.
        d_list = []
        for r_single in r_b:
            r_np = r_single.detach().cpu().numpy()
            cost_np_local = cost_detached.cpu().numpy()
            race_np_local = race_detached.cpu().numpy()
            
            if group:
                d_np = solve_group(r_np, cost_np_local, race_np_local, Q=Q_val, alpha=alpha)
            else:
                # FIXED: Correct parameter order for solve_closed_form
                # Function signature: solve_closed_form(g, r, c, alpha, Q)
                d_np,_ = solve_closed_form(g_detached.cpu().numpy(), r_np, cost_np_local, alpha, Q_val)
                # d_np, _ = solve_optimization(g_detached.cpu().numpy(), r_np, cost_np, alpha, Q)

            
            d_list.append(torch.from_numpy(d_np).to(r_b.device, r_b.dtype))
        return torch.stack(d_list)


    # --- Differentiable update function (closure) ---
    def update_fn(r, d_star):
        # r and d_star are batched
        g_b = g_detached.expand_as(r)
        
        # Call the PGD step with the captured projection_layer and group flag
        return pgd_step_cvxpylayer(r, d_star, g_b, race_detached, cost_detached, Q, alpha, lr, projection_layer, group)

    return FoldOptLayer(solver_fn, update_fn, n_iter=n_fixedpt, backprop_rule=backprop_rule)

# Train Func

In [14]:
def train_model_differentiable_dfl(
    predmodel,
    feats_train, risk_train, gainF_train, cost_train, race_train,
    alpha, Q,
    group=True,
    num_epochs=50,
    lr_pred=1e-3,
    pgd_lr=5e-3,
    n_fixedpt=50,
    lambda_fairness=0.0,
    fairness_type='atkinson'
):
    device = next(predmodel.parameters()).device
    optimizer = torch.optim.Adam(predmodel.parameters(), lr=lr_pred)
    
    feats = torch.from_numpy(feats_train).to(device)
    risk = torch.from_numpy(risk_train).to(device)
    gainF = torch.from_numpy(gainF_train).to(device)
    cost = torch.from_numpy(cost_train).to(device)
    race = torch.from_numpy(race_train).to(device)

    # --- Pre-calculate optimal solution (ground truth) ---
    if group:
        opt_d_np = solve_group(risk.cpu().numpy(), cost.cpu().numpy(), race.cpu().numpy(), Q=Q, alpha=alpha)
    else:
        opt_d_np, _ = solve_closed_form(gainF.cpu().numpy(), risk.cpu().numpy(), cost.cpu().numpy(), alpha=alpha, Q=Q)
        # NOTE: solve_closed_form sometimes returns incorrect objective values
        # We'll recalculate the objective manually for safety
    opt_d = torch.from_numpy(opt_d_np).to(device)
    
    # --- CORRECTED: Calculate optimal objective ---
    # ALWAYS recalculate objective manually to avoid numerical errors from solvers
    if group:
        opt_obj = compute_coupled_group_obj_torch(opt_d, risk, race, alpha)
    else:
        # For individual optimization, manually calculate objective using our consistent function
        opt_obj = alpha_fair_individual(opt_d * risk * gainF, alpha)


    # --- Build the FoldOpt layer with the 'group' flag ---
    differentiable_opt_layer = make_foldopt_layer_cvxpylayer(
        gainF, cost, race, alpha, Q, group, lr=pgd_lr, n_fixedpt=n_fixedpt
    )

    logs = {"loss": [], "regret": [], "fairness": []}
    
    for epoch in range(1, num_epochs + 1):
        predmodel.train()
        
        pred_risk = predmodel(feats).clamp(min=1)
        d_pred = differentiable_opt_layer(pred_risk.unsqueeze(0)).squeeze(0)
        
        # --- Loss Calculation ---
        if group:
            # Use d_pred here to keep the computation graph intact
            pred_obj = compute_coupled_group_obj_torch(d_pred, risk, race, alpha)
        else:
            # For individual optimization, ensure consistent calculation
            # IMPORTANT: Use TRUE risk values for objective evaluation, not predicted risk
            pred_obj = alpha_fair_individual(d_pred * risk * gainF, alpha)

        # Calculate normalized regret for logging
        normalized_regret = (opt_obj - pred_obj) / (torch.abs(opt_obj) + 1e-8)
        
        # Use raw regret (not normalized) for loss calculation
        raw_regret = opt_obj - pred_obj
        
        # --- Fairness Penalty ---
        fairness_penalty = torch.tensor(0.0, device=device)
        if lambda_fairness > 0:
            mode = 'between' if group else 'individual'
            if fairness_type == 'atkinson':
                fairness_penalty = atkinson_loss(pred_risk, risk, race, beta=0.5, mode=mode)
            elif fairness_type == 'mad':
                fairness_penalty = mean_abs_dev(pred_risk, risk, race, mode=mode)

        # Use raw regret for loss, but log normalized regret
        loss = raw_regret + lambda_fairness * fairness_penalty
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        logs["loss"].append(loss.item())
        logs["regret"].append(normalized_regret.item())  # Log normalized regret
        logs["fairness"].append(fairness_penalty.item())
            
    # --- Final Evaluation for Detailed Logging ---
    predmodel.eval()
    with torch.no_grad():
        final_pred_risk = predmodel(feats).clamp(min=1)
        final_d_pred = differentiable_opt_layer(final_pred_risk.unsqueeze(0)).squeeze(0)
        
        eval_logs = {}
        unique_groups = torch.unique(race).cpu().numpy()
        for g in unique_groups:
            mask = (race == g)
            if mask.sum() == 0: continue
            eval_logs[f'G{int(g)}_mse'] = (final_pred_risk[mask] - risk[mask]).pow(2).mean().item()
            group_utility = (final_d_pred[mask] * risk[mask] * gainF[mask])
            eval_logs[f'G{int(g)}_decision_obj'] = alpha_fair_individual(group_utility, alpha).item()
            eval_logs[f'G{int(g)}_true_benefit'] = risk[mask].mean().item()

    return predmodel, logs, eval_logs

In [15]:
def train_many_trials_foldopt(n_trials: int, base_seed: int, **train_args):
    """
    Runs `train_model_differentiable_dfl` for `n_trials` and aggregates the results.
    """
    per_trial_metrics = defaultdict(list)
    final_model = None
    
    # Extract data for splitting - these are passed in train_args
    feats = train_args.pop('feats')
    risk = train_args.pop('risk')
    gainF = train_args.pop('gainF')
    cost = train_args.pop('cost')
    race = train_args.pop('race')

    for t in range(n_trials):
        seed = base_seed + t
        torch.manual_seed(seed)
        np.random.seed(seed)
        print(f"--- Running Trial {t+1}/{n_trials} (Seed: {seed}) ---")

        # Create a new train/test split for each trial
        (feats_tr, _, risk_tr, _, gainF_tr, _, cost_tr, _, race_tr, _) = train_test_split(
            feats, risk, gainF, cost, race, test_size=0.5, random_state=seed
        )
        
        # Instantiate a new model for each trial to ensure fresh weights
        input_dim = feats_tr.shape[1]
        predictor = FairRiskPredictor(input_dim=input_dim).to(DEVICE)
        
        # Run the training
        _, logs, eval_logs = train_model_differentiable_dfl(
            predmodel=predictor,
            feats_train=feats_tr, risk_train=risk_tr, gainF_train=gainF_tr, 
            cost_train=cost_tr, race_train=race_tr,
            **train_args
        )

        # Log metrics from the final state of the trial
        if logs and 'regret' in logs:
            per_trial_metrics['regret'].append(logs['regret'][-1])
            per_trial_metrics['loss'].append(logs['loss'][-1])
            per_trial_metrics['fairness_penalty'].append(logs['fairness'][-1])
        
        if eval_logs:
            for key, value in eval_logs.items():
                per_trial_metrics[key].append(value)
        else:
            print(f"Warning: Trial {t+1} did not produce evaluation logs.")

    # Aggregate results across all trials
    avg_results = {}
    print("\n" + "="*60)
    print("        AVERAGED METRICS ACROSS ALL TRIALS")
    print("="*60)
    for key, values in per_trial_metrics.items():
        if values:
            mu, sigma = np.mean(values), np.std(values)
            avg_results[key] = mu
            avg_results[f'{key}_std'] = sigma
            print(f"[{key.upper():>25s}]   μ = {mu:.4f} | σ = {sigma:.4f}")
        else:
            avg_results[key] = np.nan
            avg_results[f'{key}_std'] = np.nan
            
    return avg_results, final_model

In [ ]:
from dataclasses import dataclass, field
import itertools
import datetime


@dataclass
class ExperimentConfigFoldOpt:
    """A dataclass to hold all experiment settings for Fold-Opt."""
    # Parameters to iterate over
    alphas = [0.5, 2.0]
    group_settings = [False, True]
    fairness_types = ['atkinson', 'mad']

    # Static problem parameters
    Q = 2500
    num_epochs = 50
    n_trials = 2
    base_seed = 2025

    # Fold-Opt specific hyperparameters
    foldopt_hparams = {
        'pgd_lr': 5e-3,
        'n_fixedpt': 50,
        'lr_pred': 1e-3,
    }

    def get_fairness_lambdas(self, fairness_type):
        """Returns the lambdas for a given fairness type."""
        if fairness_type == 'atkinson':
            return [0, 1, 5]
        if fairness_type == 'mad':
            return [0, 0.05, 0.5]
        return [0]

# ===================================================================
# 3. RESULTS PROCESSING: Renames and displays the final DataFrame
# ===================================================================

def process_and_display_results(results_df: pd.DataFrame):
    """Renames, reorders, and prints the final results DataFrame."""
    if results_df.empty:
        print("No results to display.")
        return
        
    rename_map = {
        'regret': 'Decision Regret',
        'fairness_penalty': 'Fairness Penalty',
        'G0_mse': 'G0 MSE', 'G1_mse': 'G1 MSE',
        'G0_decision_obj': 'G0 Decision Obj', 'G1_decision_obj': 'G1 Decision Obj',
        'G0_true_benefit': 'G0 True Benefit', 'G1_true_benefit': 'G1 True Benefit'
    }
    # Create mean and std versions of renames
    full_rename_map = {}
    for key, val in rename_map.items():
        full_rename_map[key] = f'{val} mean'
        full_rename_map[f'{key}_std'] = f'{val} std'
    
    results_df.rename(columns=full_rename_map, inplace=True)

    primary_cols = [
        'Group', 'Alpha', 'Lambda', 'FairnessType',
        'Decision Regret mean', 'G0 MSE mean', 'G1 MSE mean'
    ]
    existing_primary_cols = [col for col in primary_cols if col in results_df.columns]
    other_cols = sorted([c for c in results_df.columns if c not in existing_primary_cols])
    
    results_df = results_df[existing_primary_cols + other_cols]

    print("\n" + "="*120)
    print(" " * 50 + "EXPERIMENTS COMPLETE")
    print("="*120)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1200):
        print(results_df)

# ===================================================================
# 4. MAIN EXECUTION HARNESS: Manages the experiment lifecycle
# ===================================================================

def run_experiments_foldopt(config: ExperimentConfigFoldOpt):
    """Executes the main experiment loop based on the provided configuration."""
    results_list = []

    experiment_params = list(itertools.product(
        config.group_settings, config.fairness_types, config.alphas
    ))

    for group, fairness, alpha in experiment_params:
        fairness_lambdas = config.get_fairness_lambdas(fairness)
        for lam in fairness_lambdas:
            # Skip redundant runs where lambda is 0
                
            run_params = {
                'Group': group, 'Alpha': alpha, 'Lambda': lam, 'FairnessType': fairness
            }
            print("\n" + "-"*80)
            print(f"RUNNING EXPERIMENT: {run_params}")
            print("-"*80)

            # Assemble all arguments for the training function
            train_args = dict(
                # Data (will be split inside the trial runner)
                feats=feats_np, risk=risk_np, gainF=gainF_np, cost=cost_np, race=race_np,
                # Problem parameters
                alpha=alpha, Q=config.Q, group=group,
                # DFL/Fairness params
                lambda_fairness=lam, fairness_type=fairness,
                # Training loop params
                num_epochs=config.num_epochs,
                # Fold-Opt specific hparams
                **config.foldopt_hparams
            )
            
            # Run multiple trials and get aggregated results
            avg_results, _ = train_many_trials_foldopt(
                n_trials=config.n_trials, 
                base_seed=config.base_seed, 
                **train_args
            )
            
            row = {**run_params, **avg_results}
            results_list.append(row)

    results_df = pd.DataFrame(results_list)
    process_and_display_results(results_df)
    return results_df

# ===================================================================
# 5. NEURAL NETWORK EXPERIMENTS
# ===================================================================

print("="*80)
print(" " * 20 + "RUNNING NEURAL NETWORK EXPERIMENTS")
print("="*80)

# Create a configuration object
exp_config = ExperimentConfigFoldOpt()

# Set trials to reasonable number for full experiments
exp_config.n_trials = 5

# Track start time
start_time = datetime.datetime.now()
print(f"Started at: {start_time}")

# Run the Neural Network experiments
nn_results = run_experiments_foldopt(exp_config)

# Save Neural Network results
nn_results.to_csv('res-foldopt-nn.csv', index=False)
print(f"\n✅ Neural Network results saved to 'res-foldopt-nn.csv'")
print(f"NN Experiments completed at: {datetime.datetime.now()}")
print(f"NN Duration: {datetime.datetime.now() - start_time}")

print("\n" + "="*80)
print("NEURAL NETWORK EXPERIMENTS COMPLETE")
print("="*80)

                    RUNNING NEURAL NETWORK EXPERIMENTS
Started at: 2025-07-21 06:03:52.215576

--------------------------------------------------------------------------------
RUNNING EXPERIMENT: {'Group': False, 'Alpha': 0.5, 'Lambda': 0, 'FairnessType': 'atkinson'}
--------------------------------------------------------------------------------
--- Running Trial 1/5 (Seed: 2025) ---
--- Running Trial 2/5 (Seed: 2026) ---
--- Running Trial 3/5 (Seed: 2027) ---
--- Running Trial 4/5 (Seed: 2028) ---
--- Running Trial 5/5 (Seed: 2029) ---

        AVERAGED METRICS ACROSS ALL TRIALS
[                   REGRET]   μ = 0.1244 | σ = 0.0040
[                     LOSS]   μ = 1885.0956 | σ = 75.3782
[         FAIRNESS_PENALTY]   μ = 0.0000 | σ = 0.0000
[                   G0_MSE]   μ = 322.4952 | σ = 13.6493
[          G0_DECISION_OBJ]   μ = 11340.3900 | σ = 92.9200
[          G0_TRUE_BENEFIT]   μ = 12.2387 | σ = 0.2786
[                   G1_MSE]   μ = 477.3596 | σ = 21.5101
[          G1_DECI

# Results

In [ ]:
raise SystemExit("Exiting after Neural Network experiments. Remove this line to continue with other experiments.")

In [ ]:

# 5. SCRIPT ENTRY POINT & DATA LOADING
# ===================================================================

# --- Run Experiments ---

# 1. Create a configuration object
exp_config = ExperimentConfigFoldOpt()

# 2. You can easily modify the config for a specific run, for example:
# To run only the group-based model with a single alpha
# exp_config.group_settings = [True]
# exp_config.alphas = [1.5]
exp_config.n_trials = 5 # For a quick test run

# 3. Run the experiments
final_results = run_experiments_foldopt(exp_config)

final_results.head()

final_results_df[['Alpha', 'Group', 'Lambda', 'FairnessType', 'final_regret_mean']]

NameError: name 'final_results_df' is not defined

In [ ]:
# Save the final results to a CSV file
final_results.to_csv('foldopt-NN-0719.csv', index=False)